In [ ]:
import pandas as pd
import numpy as np
import statistics
import cufflinks as cf

def most_common(row):
    return max(set(row), key=row.count)

def get_seconds(row):
    return row.second

def sort_dt(row):
    return sorted(row)

def get_td_mean(row):
    td = 0
    if len(row) > 2:
        for i in range (0,len(row)-1):
            td += row[i+1]-row[i]
        return td/(len(row)-1)
    else:
        return 0
        
def get_td_sd(row):
    sd = 0
    new_list = []
    if len(row) > 2:
        for i in range (0,len(row)-1):
            new_list.append(row[i+1]-row[i])
        return statistics.stdev(new_list)
    else:
        return 0

In [ ]:
data = pd.read_csv("Actual-Data/stingar_full-20190520.csv")
data

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
data['d_time'] = pd.to_datetime(data['d_time']).values.astype(np.int64)

In [ ]:
data_grouped_td = data[['src_ip','d_time']].groupby('src_ip',as_index = False).agg({'d_time':lambda x: x.tolist()})
data_grouped_td['d_time'] = data_grouped_td['d_time'].apply(sort_dt)
data_grouped_td['d_time'] = data_grouped_td['d_time'].apply(get_td_mean)
data_grouped_td['d_time'] = pd.to_datetime(data_grouped_td['d_time'], unit='ns')
data_grouped_td['d_time'] = data_grouped_td['d_time'].apply(get_seconds)
data_grouped_td = data_grouped_td.rename({'d_time':'mean_time_difference'},axis = 1)
data_grouped_td

In [ ]:
data_grouped_sd = data[['src_ip','d_time']].groupby('src_ip',as_index = False).agg({'d_time':lambda x: x.tolist()})
data_grouped_sd['d_time'] = data_grouped_sd['d_time'].apply(sort_dt)
data_grouped_sd['d_time'] = data_grouped_sd['d_time'].apply(get_td_sd)
data_grouped_sd['d_time'] = pd.to_datetime(data_grouped_sd['d_time'], unit='ns')
data_grouped_sd['d_time'] = data_grouped_sd['d_time'].apply(get_seconds)
data_grouped_sd = data_grouped_sd.rename({'d_time':'sd_time_difference'},axis = 1)
data_grouped_sd 

In [ ]:
data_grouped = data[['src_ip','d_time']].groupby('src_ip',as_index = False).agg({'d_time':np.mean})
data_grouped['d_time'] = pd.to_datetime(data_grouped['d_time'], unit='ns')
data_grouped

In [ ]:
data_grouped1 = data[['src_ip','d_time']].groupby('src_ip',as_index = False).agg({'d_time':np.std})
data_grouped1['d_time'] = pd.to_datetime(data_grouped1['d_time'], unit='ns')
data_grouped1['d_time'] = data_grouped1['d_time'].apply(get_seconds)
data_grouped1

In [ ]:
data_grouped2 = data[['sensor','src_ip']].groupby('src_ip',as_index = False).agg({'sensor':lambda x: x.tolist()})
data_grouped2['most_common_sensor'] = data_grouped2['sensor'].apply(most_common)
data_grouped2

In [ ]:
data_final = data_grouped.merge(data_grouped1, left_on='src_ip', right_on='src_ip')
data_final = data_final.merge(data_grouped_td,left_on='src_ip', right_on='src_ip')
data_final = data_final.merge(data_grouped_sd,left_on='src_ip', right_on='src_ip')
data_final = data_final.merge(data_grouped2,left_on='src_ip', right_on='src_ip')
data_final = data_final.rename({'d_time_x':'mean_time_of_attack','d_time_y':'sd_time_of_attack','sensor':'all_sensors'},axis = 1)

In [ ]:
data_final

In [ ]:
data_final["all_sensors"].iloc[0]

In [ ]:
data_final["sensor_number"] = data_final["all_sensors"].apply(len)
data_final

In [ ]:
data_final.drop(["mean_time_of_attack", "sd_time_of_attack"], axis = 1)

In [ ]:
data_final.drop(["mean_time_of_attack", "sd_time_of_attack", "all_sensors", "most_common_sensor"], axis = 1, inplace= True)

In [ ]:
data_final

In [ ]:
new = data[["ssh_username", "src_ip"]].dropna()

In [ ]:
new["length_username"] = new["ssh_username"].apply(len)

In [ ]:
user_length = new.groupby("src_ip").mean()

In [ ]:
data_final

In [ ]:
current = pd.merge(user_length, data_final, how = "outer", on = "src_ip")

In [ ]:
current

In [ ]:
current['length_username'].fillna(value = current['length_username'].mean(), inplace = True)

In [ ]:
current

In [ ]:
new_command = data[['src_ip', 'command']]
new_command.dropna(inplace = True)
new_command['length_command'] = new_command['command'].apply(len) 

In [ ]:
feature = new_command.groupby('src_ip').mean()

In [ ]:
current = pd.merge(current, feature, how = 'outer', on = "src_ip")

In [ ]:
current

In [ ]:
current['length_command'].fillna(value = current['length_command'].mean(), inplace = True)

In [ ]:
current

In [ ]:
data['app'].value_counts()

In [ ]:
counts = data['app'].value_counts()
res = data[~data['app'].isin(counts[counts < 27].index)]
res['app'].value_counts()
honeypot = res[['app', 'src_ip']]
honeypot

In [ ]:
current

In [ ]:
honeypot['app'].value_counts()
honeypot.groupby('src_ip').head()

In [ ]:
new_current = pd.merge(current, honeypot, how = 'inner', on = 'src_ip')
current

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [ ]:
current

In [ ]:
dat = data.groupby('src_ip')[['src_ip', 'app']].head()
temp = dat.drop_duplicates()

In [ ]:
let = pd.get_dummies(temp['app'])
let.head()

In [ ]:
det = pd.concat([let,temp], axis = 1)
det['app'].value_counts()
counts = det['app'].value_counts()
det = det[~det['app'].isin(counts[counts < 100].index)]
det['app'].value_counts()

In [ ]:
current.columns

In [ ]:
current.drop_duplicates('src_ip', inplace = True)

In [ ]:
current

In [ ]:
head = data.groupby('src_ip').count()
head.head()
head.reset_index(inplace = True)
val = head[['src_ip', 'app']]
val.head()

In [ ]:
current = pd.merge(current, val, how = 'inner', on = 'src_ip')

In [ ]:
current.rename(columns={'app':'daily_frequency'}, inplace=True)

In [ ]:
current

In [ ]:
new = data[['src_ip', 'ssh_password']]
new.head()

In [ ]:
new_data = data[['src_ip', 'ssh_password']]
new_data.head()
new_data.dropna(inplace = True)

In [ ]:
new_data['length_password'] = new_data['ssh_password'].apply(len)

In [ ]:
new_data.drop(['ssh_password'], axis = 1, inplace = True)

In [ ]:
new_data.drop_duplicates(inplace= True)

In [ ]:
new_data

In [ ]:
current = pd.merge(current, new_data, how = 'outer', on = 'src_ip')
current['length_password'].fillna(value = current['length_password'].mean(), inplace = True)

In [ ]:
current.drop_duplicates(inplace= True)
current.drop_duplicates('src_ip', inplace= True)
current

In [ ]:
dat_t = data.groupby('src_ip')['dest_port'].nunique()
new_dat_t = dat_t.reset_index()
new_dat_t

In [ ]:
current = pd.merge(new_dat_t, current, how = 'inner', on = 'src_ip')

In [ ]:
current['dest_port'].value_counts()

In [ ]:
current.rename(columns={'dest_port':'dest_port_number'}, inplace=True)

In [ ]:
current['length_command'].value_counts()

In [ ]:
current['length_password'].value_counts()

In [ ]:
current['length_username'].value_counts()